# A simple elastic behaviour

## `MFront` implementation

To give a first introduction to writing material behaviors using `MFront`, we will implement a very simple linear elastic model :

$$
\tens{\sigma} = \lambda\:\mathrm{tr}(\tens{\varepsilon})\tens{I}+2\mu\tens{\varepsilon},
$$

where $\lambda$ and $\mu$ are the Lamé coefficients.
The parameters of the model will be Young's modulus $E$ and Poisson's ration $\nu$, while the output will be the Cauchy stress tensor $\tens{\sigma}$.
Of course, we could use one of `Mfront`'s standard `brick` for this purpose, but the idea here is to define everything ourselves.

Here is a look at the corresponding `elasticity.mfront` file:

```cpp
@DSL DefaultGenericBehaviour;
@Author Maxime Pierre;
@Date 22/11/2025;

@Behaviour Elasticity;
```

First, we specify the _domain specific language_ (DSL) that we want to use.
It will condition what `keywords` we will have access to in the rest of the file.
After optionally declaring an `author` and a `date`, we give a name to our behaviour (`Elasticity`).


Next, we define the input and output variables of our model : the strain tensor $\tens{\varepsilon}$ (a `gradient` of the displacement), and its associated `flux` which is the stress tensor $\tens{\sigma}$.

```cpp
@Gradient StrainStensor εᵗᵒ;
εᵗᵒ.setGlossaryName("Strain");

@Flux StressStensor σ;
σ.setGlossaryName("Stress");
```

Then, we define the material properties which must be passed to the behaviour:

```cpp
@MaterialProperty stress E;
E.setGlossaryName("YoungModulus");

@MaterialProperty real ν;
ν.setGlossaryName("PoissonRatio");
```
:::{note} Naming variables in `MFront`
:class: dropdown
:open: true
Note that all the names of variables and properties used so far are very common, and part of the standard `glossary` of `MFront`. They are therefore applied using the `setGlossaryName` method. For names which are not part of the glossary, any name can be applied using the `setEntryName` method instead.
:::

Now, we define the blocks composing the *tangent operator*.
In our simple case, their is only one, $\partial\tens{\sigma}/\partial\tens{\varepsilon}$, which corresponds to the stiffness tensor $\mathbb{C}$.

```cpp
@TangentOperatorBlocks{∂σ∕∂Δεᵗᵒ};
@ProvidesTangentOperator ;
```

Finally, we enter the core of the behaviour with the `Integrator` block.
This is where the behaviour of the material is actually calculated.
`MFront` provides some helper functions to calculate Lamé's coefficients from $E$ and $\nu$.
Then, we calculate the stress with an incremental formulation (note that in the case of elasticity, a total formulation would also have been acceptable).
The last part gives the one tangent operator block that was declared above.

```cpp
@Integrator{
  const auto λ = computeLambda(E, ν);
  const auto μ = computeMu(E, ν);
  σ += λ ⋅ trace(Δεᵗᵒ) ⋅ I₂ + 2 ⋅ μ ⋅ Δεᵗᵒ ;

  if (computeTangentOperator_) {
    ∂σ∕∂Δεᵗᵒ = λ ⋅ (I₂ ⊗ I₂) + 2 ⋅ μ ⋅ I₄;
  }
}
```


## Testing the behaviour with `mtest`



## Interacting with the behaviour in `python`

Using the `mtest` package in `python`, it is possible to interact with a compiled `MFront` behaviour.

In [1]:
from mtest import (
    MTest,
    VerboseLevel,
    PredictionPolicy,
    setVerboseMode,
    MTestCurrentState,
    MTestWorkSpace,
    Behaviour,
)

material_properties = {"YoungModulus": 1e10, "PoissonRatio": 0.2}
test = MTest()
setVerboseMode(VerboseLevel.VERBOSE_LEVEL2)
test.setBehaviour("generic", "../MFront_library/src/libBehaviour.dylib", "Elasticity")
#behaviour = Behaviour(interface, lib_path, model_name, hypothesis)
for property, value in material_properties.items():
    test.setMaterialProperty(property, value)

test.setImposedGradient("StrainXX", 0.0)
test.setImposedGradient("StrainYY", 0.0)
test.setImposedThermodynamicForce("StressZZ", {0.0: 0.0, 1.0: -1e7})


ArgumentError: Python argument types in
    MTest.setImposedThermodynamicForce(MTest, str, dict)
did not match C++ signature:
    setImposedThermodynamicForce(mtest::MTest {lvalue}, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>> name, std::__1::map<double, double, std::__1::less<double>, std::__1::allocator<std::__1::pair<double const, double>>> values, mtest::ConstraintOptions options)
    setImposedThermodynamicForce(mtest::MTest {lvalue}, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>> name, std::__1::map<double, double, std::__1::less<double>, std::__1::allocator<std::__1::pair<double const, double>>> values)
    setImposedThermodynamicForce(mtest::MTest {lvalue}, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char>> name, double values, mtest::ConstraintOptions options)
    setImposedThermodynamicForce(mtest::MTest {lvalue}, std::__1::basic_string<char, std::__1::char_traits<c